# Aplicación del Aprendizaje Automático en la detección de pacientes con Hipotiroidismo

## Autores

* Pol Casacuberta Gil
* Marta Granero i Martí

### Carga de los datos

#### Librerías necesarias

In [ ]:
from scipy.io import arff
import pandas as pd

In [ ]:
data = arff.loadarff('hypothyroid.arff')
df = pd.DataFrame(data[0])

#Primer tratamiento del DataFrame para pasar las columnas de tipo string a formato utf8, ya que sino teníamos b'f'
columnes = df.applymap(lambda col: isinstance(col, bytes)).all(0)
columnes = df.columns[columnes]

df.loc[:, columnes] = df[columnes].applymap(lambda col: col.decode("utf-8", errors='ignore'))
df

### Estadística descriptiva

In [ ]:
#Miramos los tipos de los atributos
df.dtypes

df.head()

df.describe(include='all').T